# MODELOS 1

In [ ]:
---------- Fonte de dados ---------------
Arquivo:'G:\Meu Drive\Mestrado\Dados\consolidado_DESFECHO_V2.csv'
Criado após correção do agrupamento dos db origem
Contém todas variáveis
Processada variável DESFECHO (Variável EVOLUCAO agregada em DESFECHO e removida. registros com EVOLUCAO nulos removidos)

----------
---------- Processamentos ---------------
0 - Removidos: registros notificados em 2022 
1 - Removidos: Numero de registros menores de 18a  6975
2 - CS_RACA agrupada em RacaBranca (branco = 1, demais = 0)
----------
---------- Variáveis ---------------
VARIÁVEIS FINAIS:
'CS_SEXO', 'CS_ESCOL_N', 'FEBRE', 'TOSSE', 'DISPNEIA', 'DESC_RESP', 'SATURACAO', 'DIARREIA', 'OUTRO_SIN',  'OBESIDADE', 'OUT_MORBI', 'FADIGA', 'CL_tmp_p_inter', 'CT_IDADE', 'RacaBranca', 'CS_ZONA_CAT','DIABETES', 'CARDIOPATI'

REMOVIDAS COLINEARIDADE: 
'GARGANTA', 'ASMA', 'PNEUMOPATI',  'PERD_PALA', 'DOR_ABD', 'PERD_OLFT', 'VOMITO'

----------
---------- OBS ---------------
RacaBranca e CS_SEXO são vairáveis binárias, mas foram processadas com o dummy. Isso não impacta no modelo.

################
# GradientBoostingClassifier
################
# Melhores parâmetros: {'learning_rate': 0.1, 'max_depth': 5, 'min_samples_leaf': 1, 'min_samples_split': 5, 'n_estimators': 200}
# Melhor score: 0.7311341191629455
################
# CATBOOST
################
# Melhores parâmetros: {'depth': 8, 'iterations': 100, 'l2_leaf_reg': 5, 'learning_rate': 0.1}
# Melhor score: 0.7308828985253862
################
# XGBOOST
################
#Melhores parâmetros: {'colsample_bytree': 1, 'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 300, 'subsample': 0.8}
#Melhor score: 0.7318569701353426
################
# LGBMClassifier
################
# Melhores parâmetros: {'lambda_l1': 1.5, 'lambda_l2': 1, 'min_data_in_leaf': 30, 'num_leaves': 127, 'reg_alpha': 0.1}
# Melhor score: 0.7360495182338803
################
# RandomForestClassifier
################
# Melhores parâmetros: {'max_depth': 20, 'min_samples_split': 5, 'n_estimators': 300}
# Melhor score: 0.7255740734607775
################
# LogisticRegression
################
# Melhores parâmetros: {'C': 0.012742749857031334, 'penalty': 'l1', 'solver': 'liblinear'}
# Melhor score (acurácia): 0.7264723132337565

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import seaborn as sns

In [ ]:
# Carga do df
df = pd.read_csv('consolidado_DESFECHO_V2.csv')

# Seleção das variáveis de interesse
df = df[[ 'DESFECHO',
'ANO_SEM_NOT',
"DT_SIN_PRI",
"CS_SEXO",
"NU_IDADE_N",
"CS_RACA",
"CS_ESCOL_N",
"CS_ZONA",
"FEBRE",
"TOSSE",
"GARGANTA",
"DISPNEIA",
"DESC_RESP",
"SATURACAO",
"DIARREIA",
"VOMITO",
"OUTRO_SIN",
"FATOR_RISC",
"CARDIOPATI",
"ASMA",
"DIABETES",
"PNEUMOPATI",
"OBESIDADE",
"OUT_MORBI",
"DT_INTERNA",
"DOR_ABD",
"FADIGA",
"PERD_OLFT",
"PERD_PALA"] ]

#Removendo Registros de 2022
print('Numero de registros iniciais ', len(df))
print('Numero de registros menores de 2022 ', len(df['ANO_SEM_NOT'].astype(str).str[0:2] == "22"))
df = df[df['ANO_SEM_NOT'].astype(str).str[0:2] != "22"]
print('Numero de registros do df final ', len(df))
print()


#Removendo Registros de menores de 18a
print('Numero de registros iniciais ', len(df))
print('Numero de registros menores de 18a ', len(df[df.NU_IDADE_N < 18]))
df = df[df.NU_IDADE_N > 17]
print('Numero de registros do df final ', len(df))

In [ ]:
#####################################
#### Processamento das variáveis
#####################################
#######
#### CRIACAO VARIÁVEL 'delta_sintomas_internação'
# Criando variáveis no df
df['DT_INTERNA'] = pd.to_datetime(df['DT_INTERNA'],format='%d/%m/%Y', errors = 'coerce')
df['DT_SIN_PRI'] = pd.to_datetime(df['DT_SIN_PRI'],format='%d/%m/%Y', errors = 'coerce')
df[ 'delta_sintomas_internação'] = (df[ 'DT_INTERNA'] - df[ 'DT_SIN_PRI']).dt.days
df['CL_tmp_p_inter'] = np.where(df['delta_sintomas_internação'].between(0, 4), 'tmp_p_inter_1',
         np.where(df['delta_sintomas_internação'].between(5, 7), 'tmp_p_inter_2',
         np.where(df['delta_sintomas_internação'].between(8, 10), 'tmp_p_inter_3',
         np.where(df['delta_sintomas_internação'].between(11, 19), 'tmp_p_inter_4',
         'tmp_p_inter_5'))))
df = df.drop(columns=(['DT_INTERNA', 'DT_SIN_PRI','delta_sintomas_internação']))
#######
## CS_SEXO
##
# M - 0
# H - 1
# I - 1 (sao 11 registros)

df['CS_SEXO'] = df['CS_SEXO'].replace({'H': 1, 'I': 1, 'F': 0, 'M': 0})
#####
## NU_IDADE_N
## Criação da categoria CT_IDADE


# Criação de bins para categorizar as idades
# Definindo os intervalos (bins)
bins = [0, 17, 49, 59, 69, 79, 140]

# Definindo os rótulos para os intervalos
labels = ['0-17', '18-49', '50-59', '60-69', '70-79', '80-140']

# Agrupando os dados na coluna 'valor_contínuo' nos intervalos definidos
df['CT_IDADE'] = pd.cut(df['NU_IDADE_N'], bins=bins, labels=labels, right=True)

df = df.drop(columns=('NU_IDADE_N'))
#####
## CS_RACA
## Criação da categoria RacaBranca
var = 'CS_RACA'
df[var] = df[var].fillna(9)
df['RacaBranca'] = np.where(df[var] == 1, 1,0)
df = df.drop(columns=(var))
#####
## CS_ESCOL_N
## criada categoria para nulos
df['CS_ESCOL_N'] = df['CS_ESCOL_N'].fillna(8)
#Categorizaçao da variávvel CS_ZONA
#8 – NULO
#1 – URBANO
#2 - RURAL, PERIRURAL, IGNORALDO

df['CS_ZONA'] = df['CS_ZONA'].fillna(8)
df['CS_ZONA_CAT'] = np.where( df['CS_ZONA']== 1, 1 , np.where(df['CS_ZONA']== 8, 0, 2))
df = df.drop(columns=('CS_ZONA'))
# Padronização das variáveis de sintomas e fatores de risco
#0 – NULO
#1 – Sim
#2 - Não
#9 - IGNORALDO
for var in ['FEBRE', "TOSSE", "GARGANTA", "DISPNEIA", "DESC_RESP","SATURACAO", "DIARREIA", "VOMITO", "OUTRO_SIN", "ASMA", "DIABETES",  "CARDIOPATI", "PNEUMOPATI", "OBESIDADE","OUT_MORBI", "DOR_ABD", "FADIGA", "PERD_OLFT", "PERD_PALA"]:
  df[var] = df[var].fillna(0)
  df[var] = df[var].replace({9: 0})
# Padronização da FATOR_RISC
#1 – Sim
#2 - Não
#9 - IGNORALDO
var = 'FATOR_RISC'
df[var] = df[var].replace({2: 0, 'S': 1, 'N':0 })

In [ ]:
################
### Avaliação de Colinearidade

corr = df.corr()

# Gera um mapa de calor
plt.figure(figsize=(25, 25))
sns.heatmap(corr, annot=True, fmt=".2f", cmap="coolwarm", cbar=True, square=True, linewidths=.5)

plt.title('Mapa de Calor da Colinearidade entre Variáveis')
plt.show()

In [ ]:
# Definidas vairáveis finais removendo as de alta colinearidade
df = df.drop(columns=['GARGANTA', 'ASMA', 'PNEUMOPATI',  'PERD_PALA', 'DOR_ABD', 'PERD_OLFT', 'VOMITO'])

In [ ]:
################
### Criação das variáveis Dummy
### NOTA - CS_SEXO e RacaBranca são variáveis binarias, podem ser removidas desse processamento.
df = pd.get_dummies(df, columns=['CS_SEXO', 'CS_ESCOL_N', 'FEBRE', 'TOSSE', 'DISPNEIA', 'DESC_RESP', 'SATURACAO', 'DIARREIA', 'OUTRO_SIN',  'OBESIDADE', 'OUT_MORBI', 'FADIGA', 'CL_tmp_p_inter', 'CT_IDADE', 'RacaBranca', 'CS_ZONA_CAT',  "DIABETES", "CARDIOPATI"], drop_first=True)

### Salvando banco de dados do modelo 2
df.to_csv('G:\Meu Drive\Mestrado\Dados\pre_modelagem.csv', index=False)

In [ ]:
################
### Carga das bibliotecas para treinamento do modelo

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
!pip install catboost
from catboost import CatBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
!pip install xgboost
import xgboost as xgb
!pip install lightgbm
import lightgbm as lgb
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

In [ ]:
################
### Otimização de hiperparametros
################
###
# Preparação do df
# Removendo a coluna ANO_SEM_NOT Não será usada no modelo geral.
df = df.drop(columns=('ANO_SEM_NOT'))
df.FATOR_RISC = df.FATOR_RISC.astype(int)
#
###

###
# Divisão em treino e teste
X_train, X_test, y_train, y_test = train_test_split(
    df.drop(columns=['DESFECHO']), # ou as colunas de features que você quer usar
    df['DESFECHO'], # a coluna alvo
    test_size=0.2, # define a proporção do teste para 20%
    random_state=42 # define a semente de aleatorização
)
#
###

################
# GradientBoostingClassifier
################
# Melhores parâmetros: {'learning_rate': 0.1, 'max_depth': 5, 'min_samples_leaf': 1, 'min_samples_split': 5, 'n_estimators': 200}
# Melhor score: 0.7311341191629455
################


param_grid = {
    'n_estimators': [100, 200],
    'learning_rate': [0.01, 0.1],
    'max_depth': [3, 5],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2]
}

gb_clf = GradientBoostingClassifier(random_state=42)
grid_search = GridSearchCV(estimator=gb_clf, param_grid=param_grid, scoring='accuracy', cv=3, verbose=1, n_jobs=-1)
grid_search.fit(X_train, y_train)

# Exibe os melhores parâmetros encontrados
print('Melhores parâmetros GradientBoostingClassifier:', grid_search.best_params_)

# Exibe o melhor score
print('Melhor score GradientBoostingClassifier:', grid_search.best_score_)

################
# CATBOOST
################
# Melhores parâmetros: {'depth': 8, 'iterations': 100, 'l2_leaf_reg': 5, 'learning_rate': 0.1}
# Melhor score: 0.7308828985253862
################
# Define o modelo
model = CatBoostClassifier()

# Define o espaço de busca dos hiperparâmetros
param_grid = {
    'depth': [4, 6, 8],
    'learning_rate': [0.01, 0.05, 0.1],
    'iterations': [30, 50, 100],
    'l2_leaf_reg': [1, 3, 5]
}

# Configura a busca em grade
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=3, scoring='accuracy', verbose=1)

# Executa a busca em grade
grid_search.fit(X_train, y_train)

# Exibe os melhores parâmetros encontrados
print('Melhores parâmetros CatBoostClassifier:', grid_search.best_params_)

# Exibe o melhor score
print('Melhor score CatBoostClassifier:', grid_search.best_score_)

################
# XGBOOST
################
#Melhores parâmetros: {'colsample_bytree': 1, 'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 300, 'subsample': 0.8}
#Melhor score: 0.7318569701353426
################
# Modelo base
model = xgb.XGBClassifier(use_label_encoder=False, eval_metric='logloss')

# Espaço de hiperparâmetros para a busca
param_grid = {
    'max_depth': [3, 4, 5],
    'learning_rate': [0.01, 0.1, 0.2],
    'n_estimators': [100, 200, 300],
    'subsample': [0.8, 1],
    'colsample_bytree': [0.8, 1],
}

# Configuração do GridSearchCV
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=3, scoring='accuracy', verbose=1)

# Executa a busca em grade
grid_search.fit(X_train, y_train)

# Melhores parâmetros e melhor score
print("Melhores parâmetros XGBClassifier:", grid_search.best_params_)
print('Melhor score XGBClassifier:', grid_search.best_score_)


################
# LGBMClassifier
################
# Melhores parâmetros: {'lambda_l1': 1.5, 'lambda_l2': 1, 'min_data_in_leaf': 30, 'num_leaves': 127, 'reg_alpha': 0.1}
# Melhor score: 0.7360495182338803
################

# Espaço de hiperparâmetros para a busca
param_grid = {
    'num_leaves': [31, 127],
    'reg_alpha': [0.1, 0.5],
    'min_data_in_leaf': [30, 50, 100],
    'lambda_l1': [0, 1, 1.5],
    'lambda_l2': [0, 1]
}
# Modelo base
gbm = lgb.LGBMClassifier(boosting_type='gbdt', objective='binary', random_state=42)
# Configuração do GridSearchCV
grid_search = GridSearchCV(estimator=gbm, param_grid=param_grid, scoring='accuracy', cv=3, verbose=1, n_jobs=-1)
# Executa a busca em grade
grid_search.fit(X_train, y_train)

# Exibe os melhores parâmetros encontrados
print('Melhores parâmetros LGBMClassifier:', grid_search.best_params_)

# Exibe o melhor score
print('Melhor score LGBMClassifier:', grid_search.best_score_)

################
# RandomForestClassifier
################
# Melhores parâmetros: {'max_depth': 20, 'min_samples_split': 5, 'n_estimators': 300}
# Melhor score: 0.7255740734607775
################


# Espaço de hiperparâmetros para a busca
param_grid = {
    'n_estimators': [200, 300],
    'max_depth': [10, 20],
    'min_samples_split': [2, 5]
}
# Modelo base
rf_clf = RandomForestClassifier(random_state=42)
# Configuração do GridSearchCV
grid_search = GridSearchCV(estimator=rf_clf, param_grid=param_grid, scoring='accuracy', cv=3, verbose=1, n_jobs=-1)
# Executa a busca em grade
grid_search.fit(X_train, y_train)

# Exibe os melhores parâmetros encontrados
print('Melhores parâmetros RandomForestClassifier:', grid_search.best_params_)

# Exibe o melhor score
print('Melhor score RandomForestClassifier:', grid_search.best_score_)


################
# LogisticRegression
################
# Melhores parâmetros: {'C': 0.012742749857031334, 'penalty': 'l1', 'solver': 'liblinear'}
# Melhor score (acurácia): 0.7264723132337565
#
################
# Definindo o espaço de parâmetros
param_grid = {
    'C': np.logspace(-4, 4, 20),  # Varia de 10^-4 a 10^4
    'penalty': ['l1', 'l2'],  # L1 é Lasso, L2 é Ridge
    'solver': ['liblinear']  # 'liblinear' é uma boa escolha para pequenos datasets e quando se usa a regularização L1
}

# Instanciando o modelo de regressão logística
log_reg = LogisticRegression()

# Instanciando o GridSearchCV
grid_search = GridSearchCV(estimator=log_reg, param_grid=param_grid, scoring='accuracy', cv=3, n_jobs=-1, verbose=1)

# Ajustando o GridSearchCV aos dados de treino
# Substitua X_train e y_train pelos seus dados de treinamento
grid_search.fit(X_train, y_train)

# Exibindo os melhores parâmetros e o melhor score
print("Melhores parâmetros LogisticRegression:", grid_search.best_params_)
print("Melhor score (acurácia) LogisticRegression:", grid_search.best_score_)

In [ ]:
########
## Comparação do desempenho dos modelos.

!pip install xgboost
!pip install catboost
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from catboost import CatBoostClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_curve, auc

# Criação de df treino e teste
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    df.drop(columns=['DESFECHO']), # ou as colunas de features que você quer usar
    df['DESFECHO'], # a coluna alvo
    test_size=0.2, # define a proporção do teste para 20%
    random_state=42) # define a semente de aleatorização

# Convertendo y_test para um array do NumPy para evitar problemas de indexação
y_test = y_test.to_numpy() if hasattr(y_test, 'to_numpy') else y_test

# Modelos
catboost_model = CatBoostClassifier(depth=8, iterations=100, l2_leaf_reg=5, learning_rate=0.1, verbose=0)
xgboost_model = XGBClassifier(colsample_bytree=1, learning_rate=0.1, max_depth=5, n_estimators=300, subsample=0.8)
gradientboost = GradientBoostingClassifier(learning_rate= 0.1, max_depth= 5, min_samples_leaf = 1, min_samples_split= 5, n_estimators= 200)
randomforest_model = RandomForestClassifier(max_depth=20, min_samples_split=5, n_estimators=300)
logistic_model = LogisticRegression(C=0.012742749857031334, penalty='l1', solver='liblinear')


models = [catboost_model, xgboost_model, gradientboost, randomforest_model, logistic_model]
model_names = ["CatBoost", "XGBoost", 'GradientBoost', "RandomForest", "LogisticRegression"]

plt.figure(figsize=(10, 8))

for model, name in zip(models, model_names):
    model.fit(X_train, y_train)
    y_pred_proba = model.predict_proba(X_test)[:, 1]
    fpr, tpr, _ = roc_curve(y_test, y_pred_proba)
    roc_auc = auc(fpr, tpr)
    plt.plot(fpr, tpr, label=f'{name} AUC = {roc_auc:.2f}')
    
    # Bootstrap para calcular o intervalo de confiança
    n_bootstraps = 1000
    bootstrapped_scores = []
    
    rng = np.random.RandomState(42)  # Reproducibilidade
    for i in range(n_bootstraps):
        # Bootstrap por amostragem com reposição nos índices dos dados de teste
        indices = rng.randint(0, len(y_pred_proba), len(y_pred_proba))
        if len(np.unique(y_test[indices])) < 2:
            # Se após a amostragem, há menos de duas classes, pule esta iteração
            continue

        score = roc_auc_score(y_test[indices], y_pred_proba[indices])
        bootstrapped_scores.append(score)
        
    sorted_scores = np.array(bootstrapped_scores)
    sorted_scores.sort()
    
    # Calculando a confiança de 95%
    confidence_lower = sorted_scores[int(0.025 * len(sorted_scores))]
    confidence_upper = sorted_scores[int(0.975 * len(sorted_scores))]
    
    print(f"{name}: AUC = {roc_auc:.3f} (95% CI: {confidence_lower:.3f} - {confidence_upper:.3f})")

plt.plot([0, 1], [0, 1], 'k--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curves')
plt.legend(loc="lower right")
plt.show()

# MODELOS 2

In [ ]:
---------- Fonte de dados ---------------
Arquivo:'G:\Meu Drive\Mestrado\Dados\consolidado_DESFECHO_V2.csv'
Criado após correção do agrupamento dos db origem
Contém todas variáveis
Processada variável DESFECHO (Variável EVOLUCAO agregada em DESFECHO e removida. registros com EVOLUCAO nulos removidos)

----------
---------- Processamentos ---------------
0 - Removidos: registros notificados em 2022 
1 - Removidos: Numero de registros menores de 18a  6975
2 - CS_RACA agrupada em RacaBranca (branco = 1, demais = 0)
----------
---------- Variáveis ---------------
VARIÁVEIS FINAIS:
'CS_SEXO', 'CS_ESCOL_N', 'FEBRE', 'TOSSE', 'DISPNEIA', 'DESC_RESP', 'SATURACAO', 'DIARREIA', 'OUTRO_SIN',  'OBESIDADE', 'OUT_MORBI', 'FADIGA', 'CL_tmp_p_inter', 'CT_IDADE', 'RacaBranca', 'CS_ZONA_CAT'

REMOVIDAS COLINEARIDADE: 
'GARGANTA', 'ASMA', 'PNEUMOPATI',  'PERD_PALA', 'DOR_ABD', 'PERD_OLFT', 'VOMITO', 'DIABETES', 'CARDIOPATI'

----------
---------- OBS ---------------
RacaBranca e CS_SEXO são vairáveis binárias, mas foram processadas com o dummy. Isso não impacta no modelo.

################
# CATBOOST
################
# Melhores parâmetros CatBoostClassifier: {'depth': 8, 'iterations': 100, 'l2_leaf_reg': 3, 'learning_rate': 0.1}
# Melhor score CatBoostClassifier: 0.7305036963486918
################

################
# XGBOOST
################
#Melhores parâmetros: {'colsample_bytree': 1, 'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 300, 'subsample': 0.8}
#Melhor score: 0.7318569701353426
################

################
# LGBMClassifier
################
# Melhores parâmetros GradientBoostingClassifier: {'learning_rate': 0.1, 'max_depth': 5, 'min_samples_leaf': 2, 'min_samples_split': 2, 'n_estimators': 200}
# Melhor score GradientBoostingClassifier: 0.7304776262351679
################

################
# RandomForestClassifier
################
# Melhores parâmetros RandomForestClassifier: {'max_depth': 10, 'min_samples_split': 5, 'n_estimators': 200}
# Melhor score RandomForestClassifier: 0.7252304225924031
################


################
# LogisticRegression
################
# Melhores parâmetros LogisticRegression: {'C': 0.03359818286283781, 'penalty': 'l1', 'solver': 'liblinear'}
# Melhor score (acurácia) LogisticRegression: 0.7261736861897804
################

In [38]:
# Carga do df
df = pd.read_csv('G:\Meu Drive\Mestrado\Dados\consolidado_DESFECHO_V2.csv')

# Seleção das variáveis de interesse
df = df[[ 'DESFECHO',
'ANO_SEM_NOT',
"DT_SIN_PRI",
"CS_SEXO",
"NU_IDADE_N",
"CS_RACA",
"CS_ESCOL_N",
"CS_ZONA",
"FEBRE",
"TOSSE",
"GARGANTA",
"DISPNEIA",
"DESC_RESP",
"SATURACAO",
"DIARREIA",
"VOMITO",
"OUTRO_SIN",
"FATOR_RISC",
"CARDIOPATI",
"ASMA",
"DIABETES",
"PNEUMOPATI",
"OBESIDADE",
"OUT_MORBI",
"DT_INTERNA",
"DOR_ABD",
"FADIGA",
"PERD_OLFT",
"PERD_PALA"] ]

#Removendo Registros de 2022
print('Numero de registros iniciais ', len(df))
print('Numero de registros menores de 2022 ', len(df['ANO_SEM_NOT'].astype(str).str[0:2] == "22"))
df = df[df['ANO_SEM_NOT'].astype(str).str[0:2] != "22"]
print('Numero de registros do df final ', len(df))
print()


#Removendo Registros de menores de 18a
print('Numero de registros iniciais ', len(df))
print('Numero de registros menores de 18a ', len(df[df.NU_IDADE_N < 18]))
df = df[df.NU_IDADE_N > 17]
print('Numero de registros do df final ', len(df))

In [41]:
#####################################
#### Processamento das variáveis
#####################################
#######
#### CRIACAO VARIÁVEL 'delta_sintomas_internação'
# Criando variáveis no df
df['DT_INTERNA'] = pd.to_datetime(df['DT_INTERNA'],format='%d/%m/%Y', errors = 'coerce')
df['DT_SIN_PRI'] = pd.to_datetime(df['DT_SIN_PRI'],format='%d/%m/%Y', errors = 'coerce')
df[ 'delta_sintomas_internação'] = (df[ 'DT_INTERNA'] - df[ 'DT_SIN_PRI']).dt.days
df['CL_tmp_p_inter'] = np.where(df['delta_sintomas_internação'].between(0, 4), 'tmp_p_inter_1',
         np.where(df['delta_sintomas_internação'].between(5, 7), 'tmp_p_inter_2',
         np.where(df['delta_sintomas_internação'].between(8, 10), 'tmp_p_inter_3',
         np.where(df['delta_sintomas_internação'].between(11, 19), 'tmp_p_inter_4',
         'tmp_p_inter_5'))))
df = df.drop(columns=(['DT_INTERNA', 'DT_SIN_PRI','delta_sintomas_internação']))
#######
## CS_SEXO
##
# M - 0
# H - 1
# I - 1 (sao 11 registros)

df['CS_SEXO'] = df['CS_SEXO'].replace({'H': 1, 'I': 1, 'F': 0, 'M': 0})
#####
## NU_IDADE_N
## Criação da categoria CT_IDADE


# Criação de bins para categorizar as idades
# Definindo os intervalos (bins)
bins = [0, 17, 49, 59, 69, 79, 140]

# Definindo os rótulos para os intervalos
labels = ['0-17', '18-49', '50-59', '60-69', '70-79', '80-140']

# Agrupando os dados na coluna 'valor_contínuo' nos intervalos definidos
df['CT_IDADE'] = pd.cut(df['NU_IDADE_N'], bins=bins, labels=labels, right=True)

df = df.drop(columns=('NU_IDADE_N'))
#####
## CS_RACA
## Criação da categoria RacaBranca
var = 'CS_RACA'
df[var] = df[var].fillna(9)
df['RacaBranca'] = np.where(df[var] == 1, 1,0)
df = df.drop(columns=(var))
#####
## CS_ESCOL_N
## criada categoria para nulos
df['CS_ESCOL_N'] = df['CS_ESCOL_N'].fillna(8)
#Categorizaçao da variávvel CS_ZONA
#8 – NULO
#1 – URBANO
#2 - RURAL, PERIRURAL, IGNORALDO

df['CS_ZONA'] = df['CS_ZONA'].fillna(8)
df['CS_ZONA_CAT'] = np.where( df['CS_ZONA']== 1, 1 , np.where(df['CS_ZONA']== 8, 0, 2))
df = df.drop(columns=('CS_ZONA'))
# Padronização das variáveis de sintomas e fatores de risco
#0 – NULO
#1 – Sim
#2 - Não
#9 - IGNORALDO
for var in ['FEBRE', "TOSSE", "GARGANTA", "DISPNEIA", "DESC_RESP","SATURACAO", "DIARREIA", "VOMITO", "OUTRO_SIN", "ASMA", "DIABETES",  "CARDIOPATI", "PNEUMOPATI", "OBESIDADE","OUT_MORBI", "DOR_ABD", "FADIGA", "PERD_OLFT", "PERD_PALA"]:
  df[var] = df[var].fillna(0)
  df[var] = df[var].replace({9: 0})
# Padronização da FATOR_RISC
#1 – Sim
#2 - Não
#9 - IGNORALDO
var = 'FATOR_RISC'
df[var] = df[var].replace({2: 0, 'S': 1, 'N':0 })

In [ ]:
################
### Remoçao do índice ano_semana

df = df.drop(columns=('ANO_SEM_NOT'))

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import seaborn as sns

In [ ]:
################
### Avaliação de Colinearidade

corr = df.corr()

# Gera um mapa de calor
plt.figure(figsize=(25, 25))
sns.heatmap(corr, annot=True, fmt=".2f", cmap="coolwarm", cbar=True, square=True, linewidths=.5)

plt.title('Mapa de Calor da Colinearidade entre Variáveis')
plt.show()

In [42]:
# Definidas vairáveis finais removendo as de alta colinearidade
df = df.drop(columns=['GARGANTA', 'ASMA', 'PNEUMOPATI',  'PERD_PALA', 'DOR_ABD', 'PERD_OLFT', 'VOMITO', "DIABETES", "CARDIOPATI"])

In [43]:
################
### Criação das variáveis Dummy
### NOTA - CS_SEXO e RacaBranca são variáveis binarias, podem ser removidas desse processamento.
df = pd.get_dummies(df, columns=['CS_SEXO', 'CS_ESCOL_N', 'FEBRE', 'TOSSE', 'DISPNEIA', 'DESC_RESP', 'SATURACAO', 'DIARREIA', 'OUTRO_SIN',  'OBESIDADE', 'OUT_MORBI', 'FADIGA', 'CL_tmp_p_inter', 'CT_IDADE', 'RacaBranca', 'CS_ZONA_CAT'], drop_first=True)

### Salvando banco de dados do modelo 2
df.to_csv('G:\Meu Drive\Mestrado\Dados\pre_modelagem_modelo_2.csv', index=False)


In [ ]:
################
### Carga das bibliotecas para treinamento do modelo

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
!pip install catboost
from catboost import CatBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
!pip install xgboost
import xgboost as xgb
!pip install lightgbm
import lightgbm as lgb
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression


In [ ]:
################
### Otimização de hiperparametros
################
###
# Preparação do df
# Removendo a coluna ANO_SEM_NOT Não será usada no modelo geral.
df = df.drop(columns=('ANO_SEM_NOT'))
df.FATOR_RISC = df.FATOR_RISC.astype(int)
#
###

###
# Divisão em treino e teste
X_train, X_test, y_train, y_test = train_test_split(
    df.drop(columns=['DESFECHO']), # ou as colunas de features que você quer usar
    df['DESFECHO'], # a coluna alvo
    test_size=0.2, # define a proporção do teste para 20%
    random_state=42 # define a semente de aleatorização
)
#
###

################
# GradientBoostingClassifier
################
# Melhores parâmetros: {'learning_rate': 0.1, 'max_depth': 5, 'min_samples_leaf': 2, 'min_samples_split': 2, 'n_estimators': 200}
# Melhor score: 0.7304776262351679
################


param_grid = {
    'n_estimators': [100, 200],
    'learning_rate': [0.01, 0.1],
    'max_depth': [3, 5],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2]
}

gb_clf = GradientBoostingClassifier(random_state=42)
grid_search = GridSearchCV(estimator=gb_clf, param_grid=param_grid, scoring='accuracy', cv=3, verbose=1, n_jobs=-1)
grid_search.fit(X_train, y_train)

# Exibe os melhores parâmetros encontrados
print('Melhores parâmetros GradientBoostingClassifier:', grid_search.best_params_)

# Exibe o melhor score
print('Melhor score GradientBoostingClassifier:', grid_search.best_score_)

################
# CATBOOST
################
# Melhores parâmetros CatBoostClassifier: {'depth': 8, 'iterations': 100, 'l2_leaf_reg': 3, 'learning_rate': 0.1}
# Melhor score CatBoostClassifier: 0.7305036963486918
################
# Define o modelo
model = CatBoostClassifier()

# Define o espaço de busca dos hiperparâmetros
param_grid = {
    'depth': [4, 6, 8],
    'learning_rate': [0.01, 0.05, 0.1],
    'iterations': [30, 50, 100],
    'l2_leaf_reg': [1, 3, 5]
}

# Configura a busca em grade
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=3, scoring='accuracy', verbose=1)

# Executa a busca em grade
grid_search.fit(X_train, y_train)

# Exibe os melhores parâmetros encontrados
print('Melhores parâmetros CatBoostClassifier:', grid_search.best_params_)

# Exibe o melhor score
print('Melhor score CatBoostClassifier:', grid_search.best_score_)

################
# XGBOOST
################
#Melhores parâmetros: {'colsample_bytree': 1, 'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 300, 'subsample': 0.8}
#Melhor score: 0.7318569701353426
################
# Modelo base
model = xgb.XGBClassifier(use_label_encoder=False, eval_metric='logloss')

# Espaço de hiperparâmetros para a busca
param_grid = {
    'max_depth': [3, 4, 5],
    'learning_rate': [0.01, 0.1, 0.2],
    'n_estimators': [100, 200, 300],
    'subsample': [0.8, 1],
    'colsample_bytree': [0.8, 1],
}

# Configuração do GridSearchCV
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=3, scoring='accuracy', verbose=1)

# Executa a busca em grade
grid_search.fit(X_train, y_train)

# Melhores parâmetros e melhor score
print("Melhores parâmetros XGBClassifier:", grid_search.best_params_)
print('Melhor score XGBClassifier:', grid_search.best_score_)


################
# LGBMClassifier
################
# Melhores parâmetros GradientBoostingClassifier: {'learning_rate': 0.1, 'max_depth': 5, 'min_samples_leaf': 2, 'min_samples_split': 2, 'n_estimators': 200}
# Melhor score GradientBoostingClassifier: 0.7304776262351679
################

# Espaço de hiperparâmetros para a busca
param_grid = {
    'num_leaves': [31, 127],
    'reg_alpha': [0.1, 0.5],
    'min_data_in_leaf': [30, 50, 100],
    'lambda_l1': [0, 1, 1.5],
    'lambda_l2': [0, 1]
}
# Modelo base
gbm = lgb.LGBMClassifier(boosting_type='gbdt', objective='binary', random_state=42)
# Configuração do GridSearchCV
grid_search = GridSearchCV(estimator=gbm, param_grid=param_grid, scoring='accuracy', cv=3, verbose=1, n_jobs=-1)
# Executa a busca em grade
grid_search.fit(X_train, y_train)

# Exibe os melhores parâmetros encontrados
print('Melhores parâmetros LGBMClassifier:', grid_search.best_params_)

# Exibe o melhor score
print('Melhor score LGBMClassifier:', grid_search.best_score_)

################
# RandomForestClassifier
################
# Melhores parâmetros RandomForestClassifier: {'max_depth': 10, 'min_samples_split': 5, 'n_estimators': 200}
# Melhor score RandomForestClassifier: 0.7252304225924031
################


# Espaço de hiperparâmetros para a busca
param_grid = {
    'n_estimators': [200, 300],
    'max_depth': [10, 20],
    'min_samples_split': [2, 5]
}
# Modelo base
rf_clf = RandomForestClassifier(random_state=42)
# Configuração do GridSearchCV
grid_search = GridSearchCV(estimator=rf_clf, param_grid=param_grid, scoring='accuracy', cv=3, verbose=1, n_jobs=-1)
# Executa a busca em grade
grid_search.fit(X_train, y_train)

# Exibe os melhores parâmetros encontrados
print('Melhores parâmetros RandomForestClassifier:', grid_search.best_params_)

# Exibe o melhor score
print('Melhor score RandomForestClassifier:', grid_search.best_score_)


################
# LogisticRegression
################
# Melhores parâmetros LogisticRegression: {'C': 0.03359818286283781, 'penalty': 'l1', 'solver': 'liblinear'}
# Melhor score (acurácia) LogisticRegression: 0.7261736861897804
################
# Definindo o espaço de parâmetros
param_grid = {
    'C': np.logspace(-4, 4, 20),  # Varia de 10^-4 a 10^4
    'penalty': ['l1', 'l2'],  # L1 é Lasso, L2 é Ridge
    'solver': ['liblinear']  # 'liblinear' é uma boa escolha para pequenos datasets e quando se usa a regularização L1
}

# Instanciando o modelo de regressão logística
log_reg = LogisticRegression()

# Instanciando o GridSearchCV
grid_search = GridSearchCV(estimator=log_reg, param_grid=param_grid, scoring='accuracy', cv=3, n_jobs=-1, verbose=1)

# Ajustando o GridSearchCV aos dados de treino
# Substitua X_train e y_train pelos seus dados de treinamento
grid_search.fit(X_train, y_train)

# Exibindo os melhores parâmetros e o melhor score
print("Melhores parâmetros LogisticRegression:", grid_search.best_params_)
print("Melhor score (acurácia) LogisticRegression:", grid_search.best_score_)

In [ ]:
########
## Comparação do desempenho dos modelos.
## Pacotes
!pip install xgboost
!pip install catboost
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from catboost import CatBoostClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_curve, auc

In [ ]:
########
## Comparação do desempenho dos modelos.

# Criação de df treino e teste
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    df.drop(columns=['DESFECHO']), # ou as colunas de features que você quer usar
    df['DESFECHO'], # a coluna alvo
    test_size=0.2, # define a proporção do teste para 20%
    random_state=42) # define a semente de aleatorização

# Convertendo y_test para um array do NumPy para evitar problemas de indexação
y_test = y_test.to_numpy() if hasattr(y_test, 'to_numpy') else y_test

# Modelos
catboost_model = CatBoostClassifier(depth=8, iterations=100, l2_leaf_reg=3, learning_rate=0.1, verbose=0)
xgboost_model = XGBClassifier(colsample_bytree=1, learning_rate=0.1, max_depth=5, n_estimators=300, subsample=0.8)
gradientboost = GradientBoostingClassifier(learning_rate= 0.1, max_depth= 5, min_samples_leaf = 2, min_samples_split= 2, n_estimators= 200)
randomforest_model = RandomForestClassifier(max_depth=10, min_samples_split=5, n_estimators=200)
logistic_model = LogisticRegression(C=0.03359818286283781, penalty='l1', solver='liblinear')


models = [catboost_model, xgboost_model, gradientboost, randomforest_model, logistic_model]
model_names = ["CatBoost", "XGBoost", 'GradientBoost', "RandomForest", "LogisticRegression"]

plt.figure(figsize=(10, 8))

for model, name in zip(models, model_names):
    model.fit(X_train, y_train)
    y_pred_proba = model.predict_proba(X_test)[:, 1]
    fpr, tpr, _ = roc_curve(y_test, y_pred_proba)
    roc_auc = auc(fpr, tpr)
    plt.plot(fpr, tpr, label=f'{name} AUC = {roc_auc:.2f}')
    
    # Bootstrap para calcular o intervalo de confiança
    n_bootstraps = 1000
    bootstrapped_scores = []
    
    rng = np.random.RandomState(42)  # Reproducibilidade
    for i in range(n_bootstraps):
        # Bootstrap por amostragem com reposição nos índices dos dados de teste
        indices = rng.randint(0, len(y_pred_proba), len(y_pred_proba))
        if len(np.unique(y_test[indices])) < 2:
            # Se após a amostragem, há menos de duas classes, pule esta iteração
            continue

        score = roc_auc_score(y_test[indices], y_pred_proba[indices])
        bootstrapped_scores.append(score)
        
    sorted_scores = np.array(bootstrapped_scores)
    sorted_scores.sort()
    
    # Calculando a confiança de 95%
    confidence_lower = sorted_scores[int(0.025 * len(sorted_scores))]
    confidence_upper = sorted_scores[int(0.975 * len(sorted_scores))]
    
    print(f"{name}: AUC = {roc_auc:.3f} (95% CI: {confidence_lower:.3f} - {confidence_upper:.3f})")

plt.plot([0, 1], [0, 1], 'k--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curves')
plt.legend(loc="lower right")
plt.show()

# Modelo 3

In [ ]:
---------- Fonte de dados ---------------
Arquivo:'G:\Meu Drive\Mestrado\Dados\consolidado_DESFECHO_V2.csv'
Criado após correção do agrupamento dos db origem
Contém todas variáveis
Processada variável DESFECHO (Variável EVOLUCAO agregada em DESFECHO e removida. registros com EVOLUCAO nulos removidos)

----------
---------- Processamentos ---------------
0 - Removidos: registros notificados em 2022 
1 - Removidos: Numero de registros menores de 18a  6975
2 - CS_RACA agrupada em Raca (nulo = o, branco = 1, demais = 2)
----------
---------- Variáveis ---------------
VARIÁVEIS FINAIS:
'CS_SEXO', 'CS_ESCOL_N', 'FEBRE', 'TOSSE', 'DISPNEIA', 'DESC_RESP', 'SATURACAO', 'DIARREIA', 'OUTRO_SIN',  'OBESIDADE', 'OUT_MORBI', 'FADIGA', 'CL_tmp_p_inter', 'CT_IDADE', 'RacaBranca', 'CS_ZONA_CAT', 'DIABETES', 'CARDIOPATI'

REMOVIDAS COLINEARIDADE: 
'GARGANTA', 'ASMA', 'PNEUMOPATI',  'PERD_PALA', 'DOR_ABD', 'PERD_OLFT', 'VOMITO'

----------
---------- OBS ---------------

CS_RACA agrupada em Raca (nulo = o, branco = 1, demais = 2)
Variáveis 'DIABETES', 'CARDIOPATI' mantidas por desempenho modelo 1  melhor do que modelo 2


################
# GradientBoostingClassifier
################
# Melhores parâmetros GradientBoostingClassifier: {'learning_rate': 0.1, 'max_depth': 5, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 200}
# Melhor score GradientBoostingClassifier: 0.7317455819848048
################
################
# CATBOOST
################
# Melhores parâmetros CatBoostClassifier: {'depth': 8, 'iterations': 100, 'l2_leaf_reg': 1, 'learning_rate': 0.1}
# Melhor score CatBoostClassifier: 0.7315962714372598
################

################
# XGBOOST
################
# Melhores parâmetros XGBClassifier: {'colsample_bytree': 1, 'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 300, 'subsample': 1}
# Melhor score XGBClassifier: 0.7322195848699681
################

################
# RandomForestClassifier
################
# Melhores parâmetros RandomForestClassifier: {'max_depth': 20, 'min_samples_split': 5, 'n_estimators': 300}
# Melhor score RandomForestClassifier: 0.7268135891141441
################
################
# LogisticRegression
################
# Melhores parâmetros LogisticRegression: {'C': 0.012742749857031334, 'penalty': 'l1', 'solver': 'liblinear'}
# Melhor score (acurácia) LogisticRegression: 0.7268349194138327
################


################
# LGBMClassifier
################
# Melhores parâmetros LGBMClassifier: {'lambda_l1': 0, 'lambda_l2': 1, 'min_data_in_leaf': 100, 'num_leaves': 127, 'reg_alpha': 0.1}
# Melhor score LGBMClassifier: 0.7325632349295033
################

In [ ]:
# Carga do df
df = pd.read_csv('G:\Meu Drive\Mestrado\Dados\consolidado_DESFECHO_V2.csv')

# Seleção das variáveis de interesse
df = df[[ 'DESFECHO',
'ANO_SEM_NOT',
"DT_SIN_PRI",
"CS_SEXO",
"NU_IDADE_N",
"CS_RACA",
"CS_ESCOL_N",
"CS_ZONA",
"FEBRE",
"TOSSE",
"GARGANTA",
"DISPNEIA",
"DESC_RESP",
"SATURACAO",
"DIARREIA",
"VOMITO",
"OUTRO_SIN",
"FATOR_RISC",
"CARDIOPATI",
"ASMA",
"DIABETES",
"PNEUMOPATI",
"OBESIDADE",
"OUT_MORBI",
"DT_INTERNA",
"DOR_ABD",
"FADIGA",
"PERD_OLFT",
"PERD_PALA"] ]

#Removendo Registros de 2022
print('Numero de registros iniciais ', len(df))
print('Numero de registros menores de 2022 ', len(df['ANO_SEM_NOT'].astype(str).str[0:2] == "22"))
df = df[df['ANO_SEM_NOT'].astype(str).str[0:2] != "22"]
print('Numero de registros do df final ', len(df))
print()


#Removendo Registros de menores de 18a
print('Numero de registros iniciais ', len(df))
print('Numero de registros menores de 18a ', len(df[df.NU_IDADE_N < 18]))
df = df[df.NU_IDADE_N > 17]
print('Numero de registros do df final ', len(df))

In [30]:
#####################################
#### Processamento das variáveis
#####################################
#######
#### CRIACAO VARIÁVEL 'delta_sintomas_internação'
# Criando variáveis no df
df['DT_INTERNA'] = pd.to_datetime(df['DT_INTERNA'],format='%d/%m/%Y', errors = 'coerce')
df['DT_SIN_PRI'] = pd.to_datetime(df['DT_SIN_PRI'],format='%d/%m/%Y', errors = 'coerce')
df[ 'delta_sintomas_internação'] = (df[ 'DT_INTERNA'] - df[ 'DT_SIN_PRI']).dt.days
df['CL_tmp_p_inter'] = np.where(df['delta_sintomas_internação'].between(0, 4), 'tmp_p_inter_1',
         np.where(df['delta_sintomas_internação'].between(5, 7), 'tmp_p_inter_2',
         np.where(df['delta_sintomas_internação'].between(8, 10), 'tmp_p_inter_3',
         np.where(df['delta_sintomas_internação'].between(11, 19), 'tmp_p_inter_4',
         'tmp_p_inter_5'))))
df = df.drop(columns=(['DT_INTERNA', 'DT_SIN_PRI','delta_sintomas_internação']))
#######
## CS_SEXO
##
# M - 0
# H - 1
# I - 1 (sao 11 registros)

df['CS_SEXO'] = df['CS_SEXO'].replace({'H': 1, 'I': 1, 'F': 0, 'M': 0})
#####
## NU_IDADE_N
## Criação da categoria CT_IDADE


# Criação de bins para categorizar as idades
# Definindo os intervalos (bins)
bins = [0, 17, 49, 59, 69, 79, 140]

# Definindo os rótulos para os intervalos
labels = ['0-17', '18-49', '50-59', '60-69', '70-79', '80-140']

# Agrupando os dados na coluna 'valor_contínuo' nos intervalos definidos
df['CT_IDADE'] = pd.cut(df['NU_IDADE_N'], bins=bins, labels=labels, right=True)

df = df.drop(columns=('NU_IDADE_N'))
#####
## CS_RACA
## Criação da categoria RacaBranca
var = 'CS_RACA'
df[var] = df[var].fillna(0)
df['RacaBranca'] = np.where(df[var] == 1, 1, np.where(df[var] == 9, 3, np.where(df[var] == 0, 0, 2)))
df = df.drop(columns=(var))
#####
## CS_ESCOL_N
## criada categoria para nulos
df['CS_ESCOL_N'] = df['CS_ESCOL_N'].fillna(8)
#Categorizaçao da variávvel CS_ZONA
#8 – NULO
#1 – URBANO
#2 - RURAL, PERIRURAL, IGNORALDO

df['CS_ZONA'] = df['CS_ZONA'].fillna(8)
df['CS_ZONA_CAT'] = np.where( df['CS_ZONA']== 1, 1 , np.where(df['CS_ZONA']== 8, 0, 2))
df = df.drop(columns=('CS_ZONA'))
# Padronização das variáveis de sintomas e fatores de risco
#0 – NULO
#1 – Sim
#2 - Não
#9 - IGNORALDO
for var in ['FEBRE', "TOSSE", "GARGANTA", "DISPNEIA", "DESC_RESP","SATURACAO", "DIARREIA", "VOMITO", "OUTRO_SIN", "ASMA", "DIABETES",  "CARDIOPATI", "PNEUMOPATI", "OBESIDADE","OUT_MORBI", "DOR_ABD", "FADIGA", "PERD_OLFT", "PERD_PALA"]:
  df[var] = df[var].fillna(0)
  df[var] = df[var].replace({9: 0})
# Padronização da FATOR_RISC
#1 – Sim
#2 - Não
#9 - IGNORALDO
var = 'FATOR_RISC'
df[var] = df[var].replace({2: 0, 'S': 1, 'N':0 })

In [31]:
################
### Remoçao do índice ano_semana

df = df.drop(columns=('ANO_SEM_NOT'))

In [32]:
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import seaborn as sns

In [ ]:
################
### Avaliação de Colinearidade

corr = df.corr()

# Gera um mapa de calor
plt.figure(figsize=(25, 25))
sns.heatmap(corr, annot=True, fmt=".2f", cmap="coolwarm", cbar=True, square=True, linewidths=.5)

plt.title('Mapa de Calor da Colinearidade entre Variáveis')
plt.show()

In [33]:
# Definidas vairáveis finais removendo as de alta colinearidade
df = df.drop(columns=['GARGANTA', 'ASMA', 'PNEUMOPATI',  'PERD_PALA', 'DOR_ABD', 'PERD_OLFT', 'VOMITO'])

In [34]:
################
### Criação das variáveis Dummy
### NOTA - CS_SEXO é variável binaria, podem ser removidas desse processamento.
df = pd.get_dummies(df, columns=['CS_ESCOL_N', 'FEBRE', 'TOSSE', 'DISPNEIA', 'DESC_RESP', 'SATURACAO', 'DIARREIA', 'OUTRO_SIN',  'OBESIDADE', 'OUT_MORBI', 'FADIGA', 'CL_tmp_p_inter', 'CT_IDADE', 'RacaBranca', 'CS_ZONA_CAT'], drop_first=True)

### Salvando banco de dados do modelo 2
df.to_csv('G:\Meu Drive\Mestrado\Dados\pre_modelagem_MODELO_3', index=False)

In [ ]:
################
### Carga das bibliotecas para treinamento do modelo

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
!pip install catboost
from catboost import CatBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
!pip install xgboost
import xgboost as xgb
!pip install lightgbm
import lightgbm as lgb
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

In [ ]:
################
### Otimização de hiperparametros
################
###
# Preparação do df
# Removendo a coluna ANO_SEM_NOT Não será usada no modelo geral.
# df = df.drop(columns=('ANO_SEM_NOT'))
df.FATOR_RISC = df.FATOR_RISC.astype(int)
#
###

###
# Divisão em treino e teste
X_train, X_test, y_train, y_test = train_test_split(
    df.drop(columns=['DESFECHO']), # ou as colunas de features que você quer usar
    df['DESFECHO'], # a coluna alvo
    test_size=0.2, # define a proporção do teste para 20%
    random_state=42 # define a semente de aleatorização
)
#
###

################
# GradientBoostingClassifier
################
# Melhores parâmetros GradientBoostingClassifier: {'learning_rate': 0.1, 'max_depth': 5, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 200}
# Melhor score GradientBoostingClassifier: 0.7317455819848048
################


param_grid = {
    'n_estimators': [100, 200],
    'learning_rate': [0.01, 0.1],
    'max_depth': [3, 5],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2]
}

gb_clf = GradientBoostingClassifier(random_state=42)
grid_search = GridSearchCV(estimator=gb_clf, param_grid=param_grid, scoring='accuracy', cv=3, verbose=1, n_jobs=-1)
grid_search.fit(X_train, y_train)

# Exibe os melhores parâmetros encontrados
print('Melhores parâmetros GradientBoostingClassifier:', grid_search.best_params_)

# Exibe o melhor score
print('Melhor score GradientBoostingClassifier:', grid_search.best_score_)

################
# CATBOOST
################
# Melhores parâmetros CatBoostClassifier: {'depth': 8, 'iterations': 100, 'l2_leaf_reg': 1, 'learning_rate': 0.1}
# Melhor score CatBoostClassifier: 0.7315962714372598
################
# Define o modelo
model = CatBoostClassifier()

# Define o espaço de busca dos hiperparâmetros
param_grid = {
    'depth': [4, 6, 8],
    'learning_rate': [0.01, 0.05, 0.1],
    'iterations': [30, 50, 100],
    'l2_leaf_reg': [1, 3, 5]
}

# Configura a busca em grade
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=3, scoring='accuracy', verbose=1)

# Executa a busca em grade
grid_search.fit(X_train, y_train)

# Exibe os melhores parâmetros encontrados
print('Melhores parâmetros CatBoostClassifier:', grid_search.best_params_)

# Exibe o melhor score
print('Melhor score CatBoostClassifier:', grid_search.best_score_)

################
# XGBOOST
################
# Melhores parâmetros XGBClassifier: {'colsample_bytree': 1, 'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 300, 'subsample': 1}
# Melhor score XGBClassifier: 0.7322195848699681
################
# Modelo base
model = xgb.XGBClassifier(use_label_encoder=False, eval_metric='logloss')

# Espaço de hiperparâmetros para a busca
param_grid = {
    'max_depth': [3, 4, 5],
    'learning_rate': [0.01, 0.1, 0.2],
    'n_estimators': [100, 200, 300],
    'subsample': [0.8, 1],
    'colsample_bytree': [0.8, 1],
}

# Configuração do GridSearchCV
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=3, scoring='accuracy', verbose=1)

# Executa a busca em grade
grid_search.fit(X_train, y_train)

# Melhores parâmetros e melhor score
print("Melhores parâmetros XGBClassifier:", grid_search.best_params_)
print('Melhor score XGBClassifier:', grid_search.best_score_)


################
# LGBMClassifier
################
# Melhores parâmetros LGBMClassifier: {'lambda_l1': 0, 'lambda_l2': 1, 'min_data_in_leaf': 100, 'num_leaves': 127, 'reg_alpha': 0.1}
# Melhor score LGBMClassifier: 0.7325632349295033
################

# Espaço de hiperparâmetros para a busca
param_grid = {
    'num_leaves': [31, 127],
    'reg_alpha': [0.1, 0.5],
    'min_data_in_leaf': [30, 50, 100],
    'lambda_l1': [0, 1, 1.5],
    'lambda_l2': [0, 1]
}
# Modelo base
gbm = lgb.LGBMClassifier(boosting_type='gbdt', objective='binary', random_state=42)
# Configuração do GridSearchCV
grid_search = GridSearchCV(estimator=gbm, param_grid=param_grid, scoring='accuracy', cv=3, verbose=1, n_jobs=-1)
# Executa a busca em grade
grid_search.fit(X_train, y_train)

# Exibe os melhores parâmetros encontrados
print('Melhores parâmetros LGBMClassifier:', grid_search.best_params_)

# Exibe o melhor score
print('Melhor score LGBMClassifier:', grid_search.best_score_)

################
# RandomForestClassifier
################
# Melhores parâmetros RandomForestClassifier: {'max_depth': 20, 'min_samples_split': 5, 'n_estimators': 300}
# Melhor score RandomForestClassifier: 0.7268135891141441
################


# Espaço de hiperparâmetros para a busca
param_grid = {
    'n_estimators': [200, 300],
    'max_depth': [10, 20],
    'min_samples_split': [2, 5]
}
# Modelo base
rf_clf = RandomForestClassifier(random_state=42)
# Configuração do GridSearchCV
grid_search = GridSearchCV(estimator=rf_clf, param_grid=param_grid, scoring='accuracy', cv=3, verbose=1, n_jobs=-1)
# Executa a busca em grade
grid_search.fit(X_train, y_train)

# Exibe os melhores parâmetros encontrados
print('Melhores parâmetros RandomForestClassifier:', grid_search.best_params_)

# Exibe o melhor score
print('Melhor score RandomForestClassifier:', grid_search.best_score_)


################
# LogisticRegression
################
# Melhores parâmetros LogisticRegression: {'C': 0.012742749857031334, 'penalty': 'l1', 'solver': 'liblinear'}
# Melhor score (acurácia) LogisticRegression: 0.7268349194138327
################
# Definindo o espaço de parâmetros
param_grid = {
    'C': np.logspace(-4, 4, 20),  # Varia de 10^-4 a 10^4
    'penalty': ['l1', 'l2'],  # L1 é Lasso, L2 é Ridge
    'solver': ['liblinear']  # 'liblinear' é uma boa escolha para pequenos datasets e quando se usa a regularização L1
}

# Instanciando o modelo de regressão logística
log_reg = LogisticRegression()

# Instanciando o GridSearchCV
grid_search = GridSearchCV(estimator=log_reg, param_grid=param_grid, scoring='accuracy', cv=3, n_jobs=-1, verbose=1)

# Ajustando o GridSearchCV aos dados de treino
# Substitua X_train e y_train pelos seus dados de treinamento
grid_search.fit(X_train, y_train)

# Exibindo os melhores parâmetros e o melhor score
print("Melhores parâmetros LogisticRegression:", grid_search.best_params_)
print("Melhor score (acurácia) LogisticRegression:", grid_search.best_score_)

In [ ]:
########
## Comparação do desempenho dos modelos.
## Pacotes
!pip install xgboost
!pip install catboost
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from catboost import CatBoostClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_curve, auc

In [ ]:
########
## Comparação do desempenho dos modelos.
df.FATOR_RISC = df.FATOR_RISC.astype(int)
# Criação de df treino e teste
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    df.drop(columns=['DESFECHO']), # ou as colunas de features que você quer usar
    df['DESFECHO'], # a coluna alvo
    test_size=0.2, # define a proporção do teste para 20%
    random_state=42) # define a semente de aleatorização

# Convertendo y_test para um array do NumPy para evitar problemas de indexação
y_test = y_test.to_numpy() if hasattr(y_test, 'to_numpy') else y_test

# Modelos
catboost_model = CatBoostClassifier(depth=8, iterations=100, l2_leaf_reg=1, learning_rate=0.1, verbose=0)
xgboost_model = XGBClassifier(colsample_bytree=1, learning_rate=0.1, max_depth=5, n_estimators=300, subsample=1)
gradientboost = GradientBoostingClassifier(learning_rate= 0.1, max_depth= 5, min_samples_leaf = 1, min_samples_split= 1, n_estimators= 200)
randomforest_model = RandomForestClassifier(max_depth=20, min_samples_split=5, n_estimators=300)
logistic_model = LogisticRegression(C=0.012742749857031334, penalty='l1', solver='liblinear')


models = [catboost_model, xgboost_model, gradientboost, randomforest_model, logistic_model]
model_names = ["CatBoost", "XGBoost", 'GradientBoost', "RandomForest", "LogisticRegression"]

plt.figure(figsize=(10, 8))

for model, name in zip(models, model_names):
    model.fit(X_train, y_train)
    y_pred_proba = model.predict_proba(X_test)[:, 1]
    fpr, tpr, _ = roc_curve(y_test, y_pred_proba)
    roc_auc = auc(fpr, tpr)
    plt.plot(fpr, tpr, label=f'{name} AUC = {roc_auc:.2f}')
    
    # Bootstrap para calcular o intervalo de confiança
    n_bootstraps = 1000
    bootstrapped_scores = []
    
    rng = np.random.RandomState(42)  # Reproducibilidade
    for i in range(n_bootstraps):
        # Bootstrap por amostragem com reposição nos índices dos dados de teste
        indices = rng.randint(0, len(y_pred_proba), len(y_pred_proba))
        if len(np.unique(y_test[indices])) < 2:
            # Se após a amostragem, há menos de duas classes, pule esta iteração
            continue

        score = roc_auc_score(y_test[indices], y_pred_proba[indices])
        bootstrapped_scores.append(score)
        
    sorted_scores = np.array(bootstrapped_scores)
    sorted_scores.sort()
    
    # Calculando a confiança de 95%
    confidence_lower = sorted_scores[int(0.025 * len(sorted_scores))]
    confidence_upper = sorted_scores[int(0.975 * len(sorted_scores))]
    
    print(f"{name}: AUC = {roc_auc:.3f} (95% CI: {confidence_lower:.3f} - {confidence_upper:.3f})")

plt.plot([0, 1], [0, 1], 'k--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curves')
plt.legend(loc="lower right")
plt.show()